In [28]:
import pandas as pd
df = pd.read_csv('Best_Books_of_the_Decade__2000s__6922_books_.csv')
df = df[:10]

In [48]:
# Start writing code here...
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

urls = df["Title_URL"]

def scrapper(urls):
    t1 = time.time()
    dic = {'num_reviews': [], 'num_pages': [], 'awards': [], 'genres': [], 'series': [], 'year_published': [], 'places': []}
    for i in range(10):
        page = requests.get(urls[i])
        soup = BeautifulSoup(page.content, 'html.parser')

        reviews = soup.find('meta', itemprop = 'reviewCount').get_text().replace(',', '').strip()
        num_reviews = int(reviews[:reviews.find('\n')])
        dic['num_reviews'].append(num_reviews)

        num_pages = soup.find('span', itemprop = 'numberOfPages').get_text()
        num_pages = int(num_pages[:num_pages.find(' ')])
        dic['num_pages'].append(num_pages)

        awards = soup.find('div', itemprop = 'awards').get_text().strip()
        awards = awards.split(',')
        awards_parsed = []
        for x in awards:
            awards_parsed.append(x[:x.find(')') + 1])
        dic['awards'].append(awards_parsed)

        genres = soup.findAll('a', class_ = 'actionLinkLite bookPageGenreLink')
        genres = [x.get_text() for x in genres][:3]
        dic['genres'].append(genres)

        series = soup.find('h2', id = 'bookSeries').get_text()
        if len(series) > 1:
            series = True
        else:
            series = False
        dic['series'].append(series)

        year = soup.find('div', id = 'details').get_text().strip().split()
        year = [int(x) for x in year[year.index('Published'):year.index('Published') + 5] if x.isnumeric() and len(x) == 4]
        dic['year_published'].append(year[0])
        
        pl = soup.find('div', id = 'bookDataBox').get_text().split('\n')
        places = ''
        f = False
        for x in pl:
            if x == 'Literary Awards':
                break
            
            if f == True:
                if len(x) > 1:
                    places += x + ' '

            if x == 'Setting':
                f = True
        dic['places'].append(places)

    df2 = pd.DataFrame(dic)
    result = pd.concat([df, df2], axis=1).to_csv('Best_2000s_Sample.csv')
    t2 = time.time()
    time_total = t2 - t1
    print("Time taken: ", int(time_total), " seconds.")
    return df2


scrapper(urls)


Time taken:  40  seconds.


,num_reviews,num_pages,awards,genres,series,year_published,places
0,66759,759,[Locus Award Nominee for Best Young Adult Nove...,"[Fantasy, Young Adult, Fiction]",True,2007,"London, England Hogwarts School of Witchcraft ..."
1,173696,374,[Locus Award Nominee for Best Young Adult Book...,"[Young Adult, Fiction, Science Fiction]",True,2008,"District 12, Panem Capitol, Panem Panem (Unite..."
2,40523,652,[Locus Award Nominee for Best Young Adult Nove...,"[Fantasy, Young Adult, Fiction]",True,2006,Hogwarts School of Witchcraft and Wizardry (Un...
3,47861,734,"[Hugo Award for Best Novel (2001), Mythopoeic...","[Fantasy, Young Adult, Fiction]",True,2002,"London, England, 1994 Hogwarts School of ..."
4,43636,870,[Bram Stoker Award for Works for Young Readers...,"[Fantasy, Young Adult, Fiction]",True,2004,Hogwarts School of Witchcraft and Wizardry (Un...
5,114588,552,[National Jewish Book Award for Children’s and...,"[Historical, Historical Fiction, Fiction]",False,2006,"Molching, 1939 (Germany) Germany"
6,47610,500,"[Arthur C. Clarke Award Nominee (2005), Orang...","[Fiction, Romance, Fantasy]",False,2013,"Chicago, Illinois (United States) Illinois (Un..."
7,74893,371,[Borders Original Voices Award for Fiction (20...,"[Fiction, Historical, Historical Fiction]",False,2004,"Kabul (Afghanistan) Fremont, California (Unite..."
8,85060,451,[Orange Prize Nominee for Fiction Longlist (20...,"[Fiction, Historical, Historical Fiction]",False,2009,"Jackson, Mississippi, 1962 (United States) Jac..."
9,55175,372,[British Book Award for Best Read of the Year ...,"[Fiction, Historical, Historical Fiction]",False,2007,Afghanistan Kabul (Afghanistan) Herat (Afghani...


'abc'